In [1]:
from datasets import load_dataset

total_dataset = load_dataset("namespace-Pt/msmarco", split="dev")
total_dataset = total_dataset.rename_column("positive", "pos")
total_dataset = total_dataset.add_column("id", [str(i) for i in range(len(total_dataset))])
total_dataset[0]

{'query': 'how many years did william bradford serve as governor of plymouth colony?',
 'pos': ['William Bradford, Governor of Plymouth Colony http://en.wikipedia.org/wiki/William_Bradford_(Plymouth_Colony_governor) William Bradford (c.1590 â\x80\x93 1657) was an English Separatist leader in Leiden, Holland and in Plymouth Colony was a signatory to the Mayflower Compact. He served as Plymouth Colony Governor five times covering about thirty years between 1621 and 1657.'],
 'id': '0'}

In [ ]:
# 统计每一个pos的数量，因为有的数据有多个对应的正样本

# from collections import Counter
# def get_pos_len(data):
#     data["pos_len"] = len(data["pos"])
#     return data

# Counter(total_dataset.map(get_pos_len)["pos_len"])
# Counter({1: 6590, 2: 331, 3: 51, 4: 8})

In [2]:
instruction = "Represent this sentence for searching relevant passages: "
total_dataset = total_dataset.add_column("prompt", [instruction] * len(total_dataset))

In [ ]:
# 在pos中只获取第一个数据
# def get_one_pos(data):
#     data["pos"] = data["pos"][0]
#     return data

# total_dataset = total_dataset.map(get_one_pos)
# total_dataset[0]

# 负样本数据构造

In [ ]:
import numpy as np

np.random.seed(42)
neg_num = 10


def add_neg(ds):
    # sample negative texts
    new_col = []
    for i in range(len(ds)):
        ids = np.random.randint(0, len(ds), size=neg_num)
        while i in ids:
            ids = np.random.randint(0, len(ds), size=neg_num)
        neg = [np.random.choice(ds[i.item()]["pos"]) for i in ids]
        new_col.append(neg)
    ds = ds.add_column("neg", new_col)
    return ds


total_dataset = add_neg(total_dataset)
# add_neg(total_dataset)[0]

In [6]:
split_dataset = total_dataset.train_test_split(0.2, 0.8, seed=42)
train_dataset, test_dataset = split_dataset["train"], split_dataset["test"]
train_dataset[0]

{'query': 'how many words for an introduction manuscript',
 'pos': ['- Introduction is useful also to the journalâ\x80\x99s reviewers and editors in judging the importance of. your manuscript. An Introduction is usually  300 to 500 words, but may be more, depending on the journal and. the topic.'],
 'id': '3222',
 'prompt': 'Represent this sentence for searching relevant passages: ',
 'neg': ["Transient Global Amnesia The doctors reassured him that it was not a stroke, but rather a memory disorder called transient global amnesia. Transient global amnesia (TGA) is a type of amnesia involving the sudden, temporary disturbance in an otherwise healthy person's memory. The other main kinds of amnesia are called anterograde and retrograde amnesia. Anterograde amnesia is a type of memory loss associated with a trauma, disease, or emotional events.",
  'Cocytus Cocytus /koÊ\x8aË\x88saÉªtÉ\x99s/ or Kokytos /koÊ\x8aË\x88kaÉªtÉ\x99s/ (Ancient Greek: ÎºÏ\x89ÎºÏ\x85Ï\x84Ï\x8cÏ\x82, literally lament

In [7]:
train_dataset.to_json(f"ft_data/training_neg_{neg_num}.json")

Creating json from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

23371760